# Initial setup

In [70]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import pickle

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx

# Get old tweets
import got3

import warnings
warnings.filterwarnings("ignore")

In [15]:
# Load saved keys from key file
keys = pickle.load(open('api_key.p', 'rb'))
auth = tw.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tw.API(auth, wait_on_rate_limit=True)

# Get trending topics

In [68]:
# See available regions
# api.trends_available() # worldwide has woeid (Where On Earth IDentifier) of 1

# Get worldwide trending topics
trends1 = api.trends_place(1) # 1 is the woeid

# Extract topic name and number of tweets for each
trends = [(trend['name'], trend['tweet_volume']) for trend in trends1[0]['trends']]

# Create pandas df
trends_df = pd.DataFrame(trends, columns=['Topic', 'Count'])
trends_df.head(10)

,Topic,Count
0,#JTBC_사과해,53543.0
1,#NYGvsPHI,16523.0
2,Eli Manning,10577.0
3,#EliminaçãoAFazenda,13035.0
4,#JENNIExGentleMonster,82173.0
5,#FlyEaglesFly,14470.0
6,Lane Johnson,NaN
7,Nick Cannon,52812.0
8,Bradesco,10875.0
9,Ertz,NaN


# Use API to download tweets

In [72]:
# Download tweets based on trending topic using API (slow)

# Specify topic of interest
topic = trends_df.Topic[2] # Eli Manning
search_term = topic + ' -filter:retweets'

# Get tweets
tweets = tw.Cursor(api.search,
                   q=search_term,
                   lang="en",
                   since='2019-12-09').items(10) # Only 10 for now

all_tweets = [tweet.text for tweet in tweets]

all_tweets[:5]

['Eli Manning has 179 yards and 2 touchdown passes in the first half. Giants lead 17-3, Darius Slayton and Leonard Wi… https://t.co/kOjbhjuKTk',
 'The Patriots reportedly didn’t send anyone to film Giants-Eagles because they already have too much footage of Eli Manning.',
 'I love Eli Manning right now.  Where’s Nick Foles when you need him... oh wait he’s gone.  🤣',
 'Imagine getting torched by 2019 Eli Manning.',
 '@Toothpick_Vic76 Remember when Eli Manning was washed up?']

# GetOldTweets to download tweets
Based on: https://github.com/fajarmf10/GetOldTweets-python and https://github.com/Jefferson-Henrique/GetOldTweets-python/issues/52#issuecomment-276932885

Steps taken:
- git clone https://github.com/Jefferson-Henrique/GetOldTweets-python
- cd GetOldTweets-python
- mv got3 ~/anaconda3/lib/python3.7/site-packages
- pip install pyquery

In [73]:
# Download tweets based on GetOldTweets
# Specify trending topic of interest
numTweets = 10 # Only 10 for now
tweetCriteria = got3.manager.TweetCriteria().setQuerySearch(topic).setMaxTweets(numTweets) 

# Get tweets
tweets = got3.manager.TweetManager.getTweets(tweetCriteria)

# Create pandas df
all_tweets_df = pd.DataFrame([], columns=['ID', 'Link', 'Username', 'Text',
                                          'Date', 'Retweets', 'Favorites',
                                          'Mentions', 'Hashtags', 'Geo',
                                          'URLs', 'Author_ID'])
for i in range(numTweets):
    all_tweets_df.loc[i] = [tweets[i].id, tweets[i].permalink, tweets[i].username, tweets[i].text,
                            tweets[i].date, tweets[i].retweets, tweets[i].favorites,
                            tweets[i].mentions, tweets[i].hashtags, tweets[i].geo,
                            tweets[i].urls, tweets[i].author_id]
all_tweets_df.head()

,ID,Link,Username,Text,Date,Retweets,Favorites,Mentions,Hashtags,Geo,URLs,Author_ID
0,1204231358161145857,https://twitter.com/fuckin_boots/status/120423...,fuckin_boots,Eagles are straight up Garbage right now...the...,2019-12-09 21:48:23,0,0,,,,,436319536
1,1204231352708718592,https://twitter.com/if71522/status/12042313527...,if71522,Eli Manning effect,2019-12-09 21:48:22,0,0,,,,,3649684998
2,1204231348774420480,https://twitter.com/BadManDarian/status/120423...,BadManDarian,That Eli Manning fella is a baaaadman,2019-12-09 21:48:21,0,0,,,,,165648816
3,1204231345796308998,https://twitter.com/JohnMcLean2012/status/1204...,JohnMcLean2012,Eli Manning is pulling a Philly Special. #...,2019-12-09 21:48:21,0,0,,#MNF #NFL,,,604795045
4,1204231340952035330,https://twitter.com/tojo381/status/12042313409...,tojo381,Eli Manning strolling into the locker room at ...,2019-12-09 21:48:19,0,0,,,,,325293512
